# Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Models
from sklearn.cluster import KMeans

# Plot Styling
import seaborn as sns; 
sns.set(); 
sns.set_style("whitegrid")  

import warnings
warnings.filterwarnings('ignore')

# Pyplot
import matplotlib.pyplot as plt

# Importing the dataset

In [ ]:
customers = pd.read_csv('data/mall_customers.csv')

# Peek the Data

In [ ]:
customers.head()

In [ ]:
customers.sample(5)

In [ ]:
customers.info()

In [ ]:
customers.describe()

# Preprocessing
### checking if the dataset contains any NULL values

In [ ]:
customers.isnull().sum()

### Visualization

In [ ]:
plt.figure(1 , figsize = (20 , 6))
n = 0 
for x in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
    n += 1
    plt.subplot(1 , 3 , n)
    plt.subplots_adjust(hspace =0.5 , wspace = 0.5)
    sns.distplot(customers[x] , bins = 20)
    plt.title('Distplot of {}'.format(x), fontsize = 20)
plt.show()

In [ ]:
plt.figure(1 , figsize = (20 , 7))
n = 0 
for x in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
    for y in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
        n += 1
        plt.subplot(3 , 3 , n)
        plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
        sns.regplot(x = x , y = y , data = customers)
        plt.ylabel(y.split()[0]+' '+y.split()[1] if len(y.split()) > 1 else y )
plt.show()

In [ ]:
customers['Age'].plot.hist(figsize = (15, 6))

In [ ]:
customers['Spending Score (1-100)'].plot.hist(figsize = (15, 5))

In [ ]:
customers.columns

### Correlation

In [ ]:
sns.heatmap(customers.corr(), annot=True)
plt.show()

# Clustering With 2 Features 
### Annual Income, Spending Score

In [ ]:
X = customers.iloc[:, [3, 4]]

In [ ]:
X.columns

### Elbow Method to find the optimal number of clusters

In [ ]:
sse = []
for i in range(1, 11):
   kmeans = KMeans(n_clusters = i, init = 'k-means++').fit(X)
   sse.append(kmeans.inertia_)

plt.figure(figsize = (12 , 6))
plt.plot(range(1,11), sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');
plt.title('Elbow Method For Optimal k')
plt.show()

K = 5 is the optimal number of clusters

### K-Means Modeling

In [ ]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++')
y_kmeans = kmeans.fit_predict(X)
centroids = kmeans.cluster_centers_
print(centroids)

customers['cluster_group_2_features'] = y_kmeans # Save predictions to new column

In [ ]:
customers.cluster_group_2_features.value_counts()

In [ ]:
plt.figure(figsize = (15 , 6))
plt.scatter(X[y_kmeans == 0]['Annual Income (k$)'],X[y_kmeans == 0]['Spending Score (1-100)'],s = 100, c ='red', label = 'Standard')
plt.scatter(X[y_kmeans == 1]['Annual Income (k$)'],X[y_kmeans == 1]['Spending Score (1-100)'],s = 100, c ='blue', label = 'Careful')
plt.scatter(X[y_kmeans == 2]['Annual Income (k$)'],X[y_kmeans == 2]['Spending Score (1-100)'],s = 100, c ='green', label = 'Sensible')
plt.scatter(X[y_kmeans == 3]['Annual Income (k$)'],X[y_kmeans == 3]['Spending Score (1-100)'],s = 100, c ='cyan', label = 'Careless')
plt.scatter(X[y_kmeans == 4]['Annual Income (k$)'],X[y_kmeans == 4]['Spending Score (1-100)'],s = 100, c ='magenta', label = 'Target')
plt.scatter(centroids[:,0],centroids[:,1],s = 300, c ='yellow', label = 'Centroids')
plt.title('Clusters of customers')
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending score (1-100)')
plt.legend()
plt.show()

### Hierarchical Clustering

In [ ]:
import scipy.cluster.hierarchy as sch
sns.set(style='white')

dendrogram = sch.dendrogram(sch.linkage(X, method = 'ward'))

fig = plt.gcf()
fig.set_size_inches(15,6)
ax = plt.gca()
bounds = ax.get_xbound()
ax.plot(bounds, [175,175],'--', c='k')
ax.plot(bounds,'--', c='k')
plt.title('Dendrogam')
plt.xlabel('Customers')
plt.ylabel('Ecuclidean Distance')
plt.show()

# Clustering With 3 Features 
### Age, Annual Income, Spending Score

In [ ]:
X2 = customers[['Age' , 'Annual Income (k$)' ,'Spending Score (1-100)']].values

### Elbow Method

In [ ]:
sse = []
for i in range(1, 11):
   kmeans2 = KMeans(n_clusters = i, init = 'k-means++').fit(X2)
   sse.append(kmeans2.inertia_)
    
plt.figure(figsize = (12 , 6))
plt.plot(range(1,11), sse, 'o')
plt.plot(range(1,11), sse, '-', alpha = 0.5)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');
plt.title('Elbow Method For Optimal k')
plt.show()

### K-means Modeling

In [ ]:
kmeans2 = KMeans(n_clusters = 6, init = 'k-means++')
y_kmeans2 = kmeans2.fit_predict(X2)
centroids2 = kmeans2.cluster_centers_
print(centroids2)

customers['cluster_group_3_features'] = y_kmeans2 # Save predictions to new column

In [ ]:
customers.cluster_group_3_features.value_counts()

### Plotting Using Plotly
If you don't have Plotly yet, please write down these codes to your terminal:
- conda install -c https://conda.anaconda.org/plotly plotly
- jupyter labextension install @jupyterlab/plotly-extension

In [ ]:
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected = True)

trace1 = go.Scatter3d(
    x = customers['Age'],
    y = customers['Spending Score (1-100)'],    
    z = customers['Annual Income (k$)'],
    mode='markers',
     marker=dict(
        color = customers['cluster_group'], 
        size= 5,
        line=dict(
            color= customers['cluster_group'],
            width= 12
        ),
        opacity=0.8
     )
)
data = [trace1]
layout = go.Layout( 
    width = 700,
    height = 500,
    title= 'Clusters',
    scene = dict(
            xaxis = dict(title  = 'Age'),
            yaxis = dict(title  = 'Spending Score'),
            zaxis = dict(title  = 'Annual Income')
            )
    )

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

### Plotting Using Matplotlib 3s

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('seaborn-whitegrid')
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(customers['Spending Score (1-100)'], customers['Age'], customers['Annual Income (k$)'], c = kmeans2.labels_, marker = 'o', cmap = 'prism', s = 120, edgecolor = 'black', linewidth = 1)
    
ax.set_xlabel("Spending Score (1-100)")
ax.set_ylabel("Age")
ax.set_zlabel("Annual Income (k$)")
plt.show()

### Different Point of View

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('seaborn-whitegrid')
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(customers['Spending Score (1-100)'], customers['Age'], customers['Annual Income (k$)'], c = kmeans2.labels_, marker = 'o', cmap = 'prism', s = 120, edgecolor = 'black', linewidth = 1)
    
ax.set_xlabel("Spending Score (1-100)")
ax.set_ylabel("Age")
ax.set_zlabel("Annual Income (k$)")
ax.view_init(10, 95)
plt.show()

### Save The Predictions To CSV

In [ ]:
customers.to_csv('customers_clusters.csv',index=False, header=True)

# Radar Chart

In [ ]:
def _scale_data(data, ranges):
    (x1, x2) = ranges[0]
    d = data[0]
    return [(d - y1) / (y2 - y1) * (x2 - x1) + x1 for d, (y1, y2) in zip(data, ranges)]

class RadarChart():
    def __init__(self, fig, location, sizes, variables, ranges, n_ordinate_levels = 6):

        angles = np.arange(0, 360, 360./len(variables))

        ix, iy = location[:] ; size_x, size_y = sizes[:]
        
        axes = [fig.add_axes([ix, iy, size_x, size_y], polar = True, 
        label = "axes{}".format(i)) for i in range(len(variables))]

        _, text = axes[0].set_thetagrids(angles, labels = variables)
        
        for txt, angle in zip(text, angles):
            if angle > -1 and angle < 181:
                txt.set_rotation(angle - 90)
            else:
                txt.set_rotation(angle - 270)
        
        for ax in axes[1:]:
            ax.patch.set_visible(False)
            ax.xaxis.set_visible(False)
            ax.grid("off")
        
        for i, ax in enumerate(axes):
            grid = np.linspace(*ranges[i],num = n_ordinate_levels)
            grid_label = [""]+["{:.0f}".format(x) for x in grid[1:-1]]
            ax.set_rgrids(grid, labels = grid_label, angle = angles[i])
            ax.set_ylim(*ranges[i])
        
        self.angle = np.deg2rad(np.r_[angles, angles[0]])
        self.ranges = ranges
        self.ax = axes[0]
                
    def plot(self, data, *args, **kw):
        sdata = _scale_data(data, self.ranges)
        self.ax.plot(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def fill(self, data, *args, **kw):
        sdata = _scale_data(data, self.ranges)
        self.ax.fill(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def legend(self, *args, **kw):
        self.ax.legend(*args, **kw)
        
    def title(self, title, *args, **kw):
        self.ax.text(0.9, 1, title, transform = self.ax.transAxes, *args, **kw)

### 2 Features

In [ ]:
radar_2_feature = pd.pivot_table(customers, index = ['cluster_group_2_features'], values=['Age','Annual Income (k$)','Spending Score (1-100)'])

fig = plt.figure(figsize=(10,12))

n_clusters = len(customers.cluster_group_2_features.value_counts())

attributes = ['Age','Annual Income (k$)','Spending Score (1-100)']
ranges = [[0.01, 70], [0.01, 137], [0.01,100]]
index  = [0, 1, 2]

i_cols = 3
i_rows = 2
size_x, size_y = (1/i_cols), (1/i_rows)

for ind in range(n_clusters):
    ix = ind%3 ; iy = i_rows - ind//3
    pos_x = ix*(size_x + 0.05) ; pos_y = iy*(size_y + 0.05)            
    location = [pos_x, pos_y]  ; sizes = [size_x, size_y] 
    #______________________________________________________
    data = np.array(radar_2_feature.loc[ind])
    
    radar = RadarChart(fig, location, sizes, attributes, ranges)
    radar.plot(data, color = 'b', linewidth=2.0)
    radar.fill(data, alpha = 0.2, color = 'b')
    radar.title(title = 'cluster nº{}'.format(ind), color = 'r')
    ind += 1 

### 3 Features

In [ ]:
radar_3_feature = pd.pivot_table(customers, index = ['cluster_group_3_features'], values=['Age','Annual Income (k$)','Spending Score (1-100)'])

fig = plt.figure(figsize=(10,12))

n_clusters = len(customers.cluster_group_3_features.value_counts())

attributes = ['Age','Annual Income (k$)','Spending Score (1-100)']
ranges = [[0.01, 70], [0.01, 137], [0.01,100]]
index  = [0, 1, 2]

i_cols = 3
i_rows = 2
size_x, size_y = (1/i_cols), (1/i_rows)

for ind in range(n_clusters):
    ix = ind%3 ; iy = i_rows - ind//3
    pos_x = ix*(size_x + 0.05) ; pos_y = iy*(size_y + 0.05)            
    location = [pos_x, pos_y]  ; sizes = [size_x, size_y] 
    #______________________________________________________  
    data = np.array(radar_3_feature.loc[ind])
    
    radar = RadarChart(fig, location, sizes, attributes, ranges)
    radar.plot(data, color = 'b', linewidth=2.0)
    radar.fill(data, alpha = 0.2, color = 'b')
    radar.title(title = 'cluster nº{}'.format(ind), color = 'r')
    ind += 1 